# Исследование надежности заемщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План выполнения проекта

1. Откроем таблицу и изучим данные:
    * Выведем основную информацию о датафрейме.
0. Предобработаем данные:
    * Обработаем или удалим пропуски;
    * Обработаем аномальные значения;
    * Изменим типы данных;
    * Обработаем явные и неявные дубликаты;
    * Проведем категоризацию данных.
0. Исследуем данные и ответьте на вопросы:
    * Есть ли зависимость между количеством детей и возвратом кредита в срок?
    * Есть ли зависимость между семейным положением и возвратом кредита в срок?
    * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    * Как разные цели кредита влияют на его возврат в срок?
    * Приведем возможные причины появления пропусков в исходных данных.
0. Напишем общий вывод.

## Описание данных

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

## Импорт библиотек

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os

### 1. Откроем таблицу и изучим общую информацию о данных.
Считаем данные из csv-файла в датафрейм и сохраним в переменную data:

In [2]:
data_1 = 'data.csv'
data_2 = 'https://code.s3.yandex.net/datasets/data.csv'

if os.path.exists(data_1):
    data = pd.read_csv(data_1)
elif os.path.exists(data_2):
    data = pd.read_csv(data_2)
else:
    print('Something is wrong')

Выведем первые 10 строчек датафрейма `data` на экран:

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 2. Предобработка данных

#### Обработаем пропуски:
Выведем количество пропущенных значений для каждого столбца:

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`:

In [6]:
for types in data['income_type'].unique():
    data.loc[(data['income_type'] == types) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == types), 'total_income'].median()

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`:

In [7]:
for types in data['income_type'].unique():
    data.loc[(data['income_type'] == types) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == types), 'days_employed'].median()

Обработаем значения в столбце `days_employed`: заменим все отрицательные значения положительными с помощью метода `abs()`:

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях:

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведитем перечень уникальных значений столбца `children`:

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`:

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Убедимся, что артефакты удалены:

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Убедитесь, что все пропуски заполнены:

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода astype():

In [14]:
data['total_income'] = data['total_income'].astype(int)

#### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв:

In [15]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их:

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

Проверим:

In [18]:
data.duplicated().sum()

0

#### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [19]:
def categorize_income(income):
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`:

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

* 'операции с автомобилем';
* 'операции с недвижимостью';
* 'проведение свадьбы';
* 'получение образования'.

In [22]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории или другая ошибка'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### 3. Исследуйте данные и ответим на вопросы:
#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для проверки данной гипотезы, для каждой категории необходимо посчитать количество должников, количество людей, которые вернули кредит в срок, и соотношение должников к общему количеству людей в каждой группе:

In [24]:
credit3_1 = data.pivot_table(index = [ 'children'], values = 'debt', aggfunc = {'count','sum'}) # создадим сводную таблицу
credit3_1 = credit3_1.rename(columns={'count':'total_debtor','sum':'nice_debtor'}) # заменим название колонок sum и count на более читаемые
credit3_1['ratio'] = credit3_1['nice_debtor']/credit3_1['total_debtor'] # посчитаем соотношение должников к общему количеству людей в каждой группе
credit3_1 = credit3_1.sort_values(by='ratio', ascending=True) # отсортируем данные в поряке возрастания

In [25]:
credit3_1

,total_debtor,nice_debtor,ratio
children,,,
5,9,0,0.000000
0,14091,1063,0.075438
3,330,27,0.081818
1,4808,444,0.092346
2,2052,194,0.094542
4,41,4,0.097561


#### Вывод:
По данным сводной таблицы, можно сделать вывод о том, что зависимость между количеством детей у заемщика и возвратом кредита в срок есть. А именно, cамые ненадежные заемщики с самой высокой долей просрочек `0.097561` - это многодетные семьи с 4-мя детьми. Самые надежные заемщики с самой низкой долей просрочек по кредиту `0.075438` - это заемщики без детей.Также мы обнаружили интересную аномалию, у заемщиков с 5-ю детьми доля просрочек `0`, но явный вывод о том что это самые надежные заемщики, мы сделать не можем.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для проверки данной гипотезы, для каждой категории необходимо посчитать количество должников, количество людей, которые вернули кредит в срок, и соотношение должников к общему количеству людей в каждой группе:

In [26]:
credit3_2 = data.pivot_table(index = [ 'family_status'], values = 'debt', aggfunc = {'count','sum'}) # создадим сводную таблицу
credit3_2 = credit3_2.rename(columns={'count':'total_debtor','sum':'nice_debtor'}) # заменим название колонок sum и count на более читаемые
credit3_2['ratio'] = credit3_2['nice_debtor']/credit3_2['total_debtor'] # посчитаем соотношение должников к общему количеству людей в каждой группе
credit3_2 = credit3_2.sort_values(by='ratio', ascending=True)

In [27]:
credit3_2

,total_debtor,nice_debtor,ratio
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639


#### Вывод:

По данным сводной таблицы, можно сделать вывод о том, что зависимость между семейным положение заемщика и возвратом кредита в срок есть. Одинокие заемщики `family_status = 'Не женат / не замужем'` имеют наибольшую долю просрочек по кредиту `0.097639`. Овдовевшие супруги `family_status = 'вдовец / вдова'` имеют наименьшую долю просрочек по кредиту `0.066246`.

В предыдущем пункте  мы сделали вывод о том, что семьи без детей платят кредит исправно. Коррелируя это с выводом из данного пункта, можно сделать вывод о том, что не иметь детей недостаточно, для того чтобы являться надежным заемщиком, нужно еще и не быть одиноким (быть женатым / замужем).

####  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для проверки данной гипотезы, для каждой категории необходимо посчитать количество должников, количество людей, которые вернули кредит в срок, и соотношение должников к общему количеству людей в каждой группе:

In [28]:
credit3_3 = data.pivot_table(index = [ 'total_income_category'], values = 'debt', aggfunc = {'count','sum'}) # создадим сводную таблицу
credit3_3 = credit3_3.rename(columns={'count':'total_debtor','sum':'nice_debtor'}) # заменим название колонок sum и count на более читаемые
credit3_3['ratio'] = credit3_3['nice_debtor']/credit3_3['total_debtor'] # посчитаем соотношение должников к общему количеству людей в каждой группе
credit3_3 = credit3_3.sort_values(by='ratio', ascending=True) # отсортируем данные в поряке возрастания

In [29]:
credit3_3

,total_debtor,nice_debtor,ratio
total_income_category,,,
D,349,21,0.060172
B,5014,354,0.070602
A,25,2,0.080000
C,15921,1353,0.084982
E,22,2,0.090909


#### Вывод:

По данным сводной таблицы, можно сделать вывод о том, что зависимость между уровнем дохода и возвратом кредита в срок есть. Самая высокая доля просрочек по кредиту `0.090909` у потенциальных заемщиков из категории `'Е'` с уровнем дохода `0-30000` руб. Далее, по росту надежности, следуют заемщики из категорий `'А'` с доходом выше `1000000` руб и `'С'` с доходом `50001–200000` доля просрочек которых, составляет `0.08` руб и `0.084982` руб соответственно. Самые надежные заемщики из категории `'D'` с доходом `30001–50000` руб c долей просрочек `0.060172`. Итого, исправнее всего кредит платят заемщики с доходом `30001–50000` руб.

#### Как разные цели кредита влияют на его возврат в срок?

Для проверки данной гипотезы, для каждой категории необходимо посчитать количество должников, количество людей, которые вернули кредит в срок, и соотношение должников к общему количеству людей в каждой группе:

In [30]:
credit3_4 = data.pivot_table(index = [ 'purpose_category'], values = 'debt', aggfunc = {'count','sum'}) # создадим сводную таблицу
credit3_4 = credit3_4.rename(columns={'count':'total_debtor','sum':'nice_debtor'}) # заменим название колонок sum и count на более читаемые
credit3_4['ratio'] = credit3_4['nice_debtor']/credit3_4['total_debtor'] # посчитаем соотношение должников к общему количеству людей в каждой группе
credit3_4 = credit3_4.sort_values(by='ratio', ascending=True) # отсортируем данные в поряке возрастания

In [31]:
credit3_4

,total_debtor,nice_debtor,ratio
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


#### Вывод:

По данным сводной таблицы, можно сделать вывод о том, что самые ненадежные заемщики те, кто берут кредиты для покупки автомобиля `0.093480` и на образование `0.092528`. Далее, по росту надежности, следуют заемщики, которые берут кредиты на проведение свадьбы `0.079118`. Cамые надежные заемщики те, кто берут кредиты на операции с недвижимостью `0.072551`.

#### Приведите возможные причины появления пропусков в исходных данных.
Ответ:

Мы получили данные о пропущенных значениях каждого столбца в исходных данных используя комбинацию двух методов, а именно isna() и sum(). В двух столбцах были найдены пропущенные значения. Один из них `days_employed`(общий трудовой стаж в днях). Другой столбец с пропущенными значениями `total_income`(ежемесячный доход). Оба столбца хранят в себе количественные данные. Так как количество строк с пропущенными значениями у этих колонок одинаковое, можно предположить, что в процессе опроса, заемщики не захотел предоставить информацию о своем стаже работы и ежемесечном доходе. Таким образом, данных пропуски можно охарактеризовать как случайные. Например, отсутствие ответа на определенный вопрос анкеты может зависеть от возраста человека. Молодые охотнее отвечают на вопрос, люди более пожилого возраста скорее избегают ответа.

Но не стоит забывать и про технические ошибки, они тоже могут повлиять на их появления.

#### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.
Ответ:

Медианное значение лучше отражает выборку в случае, если есть незаполненные данные. Среднее арифметическое отражает ситуацию некорректно, так как не учитывает разброс данных.

### 4. Общий вывод:

Были предобработаны данные, а именно:
* Обработали пропущенные значения в столбцах `days_employed` и `total_income` содержащих количественные данные и заполнили пропущенные значения в них медианным значением. Пропущенные значение обнаружили комбинацие методов `isna` и `sum`;
* Обработали аномальные значение в столбцах `days_employed` и `children`. В первом случае артефактом оказались отрицательные значения количества дней трудового стажа. Все отрицательные значения заменили положительными с помощью метода `abs()`. Во втором случае были два аномальных значения `20` и `-1`, но так как количество детей не может быть отрицательным и на практике не может быть равным `20`(может конечно, но не для наших реалей), было принято решение удалить данные строки из датафрейма. Удалиение производилось путем исключения данных строк из датафрейма;
* Для дальнейшей, более удобной, обработки данных были проделаны следующие шаги:
    * Заменили вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`;
    * Обработали неявные дубликаты в столбце `education`. В этом столбце были одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру методом `lower()` и удалили строки-дубликаты в данных методом `drop_duplicates()`;
    * Провели категоризацию данных. При этом использовали собственные функции `categorize_income()`, `categorize_purpose()` и метод `apply()`. Значения категорий заполнили в столбцы `total_income_category` и `purpose_category` соответственно.
    * Далее приступили к обработке данных и проверке гипотез.
* Выяснили, что по выдвинутым гипотезам нет четкой взаимосвязи, но тем не менее, по проведенному анализу можно сделать вывод о том, что люди, которые с наибольшей вероятностью допустят просрочку по кредиту:
    * Содержат 4 ребенка (`0.097561`);
    * Состоят в гражданском браке (`0.093130`) или не состоят в браке вовсе (`0.097639`);
    * Доход которыбудет существенно низок(у заемщиков из категории `'Е'` с уровнем дохода `0-30000` руб (`0.090909`);
    * Целью кредита будет желание взять автокредит(`0.093480`) или кредит на образование(доля просрочек `0.092528`).
    * Комбинация этих факторов может говорить о том, что в будущем возможны просрочки по кредиту.
* Высока вероятность возврата кредита в срок, если человек:
    * Не состоит в браке (`0.070648`);
    * Имеет высокий доход (`0.060172`);
    * У него нет детей (`0.075438`);
    * Цель кредита - операции с недвижимостью(`0.072551`). 
    
   *В скобках указано значение доли просрочек по категориям или цели заемщика.